In [5]:
import sys

!{sys.executable} -m pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 6.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 460.3 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: packaging
    Found existing installation: packaging 19.1
    Uninstalling packaging-19.1:
      Successfully uninstalled packaging-19.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
liwc-text-analysis 1.0.2 requires packaging==19.1, but you have packaging 23.1 which is incompatible.


In [3]:
HOME_DIR = "/home_remote"
HOME = "/home/thi.tra.my.nguyen"

In [8]:
from liwc import Liwc
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
# Load LIWC dictionary
liwc = Liwc(os.path.join(HOME_DIR, "master_thesis/LIWC2007_English100131.dic"))

In [13]:
# Load data
%store -r df
%store -r df_neg



In [17]:
#combine all text of same user into one string
text_per_user_pos = df.groupby('User')['Text'].apply(' '.join).reset_index()
text_per_user_neg = df_neg.groupby('User')['Text'].apply(' '.join).reset_index()
#add label
text_per_user_pos['label'] = 1
text_per_user_neg['label'] = 0

In [25]:
# input text to LIWC
liwc_pos = [liwc.parse(text.split(' ')) for text in text_per_user_pos['Text']]
liwc_neg = [liwc.parse(text.split(' ')) for text in text_per_user_neg['Text']]
#then buid a dictionary with key as user, value as a list of LIWC features
liwc_pos_dict = {}
liwc_neg_dict = {}
for i in range(len(liwc_pos)):
    liwc_pos_dict[text_per_user_pos['User'][i]] = liwc_pos[i]
    liwc_neg_dict[text_per_user_neg['User'][i]] = liwc_neg[i]

In [29]:
#calculate percentage of each LIWC feature per user
liwc_pos_percent = {}
liwc_neg_percent = {}
for key, value in liwc_pos_dict.items():
    liwc_pos_percent[key] = {k: v / sum(value.values()) for k, v in value.items()}
for key, value in liwc_neg_dict.items():
    liwc_neg_percent[key] = {k: v / sum(value.values()) for k, v in value.items()}


In [28]:
liwc_pos_percent

{'train_subject1095': {'verb': 0.06236148521940741,
  'present': 0.03706229329332742,
  'cogmech': 0.06795322036209894,
  'insight': 0.008933137849909645,
  'funct': 0.2180435746189778,
  'article': 0.0338572743700774,
  'auxverb': 0.040199120324593406,
  'future': 0.004500664870946845,
  'money': 0.005523543250707491,
  'preps': 0.05550820007501108,
  'leisure': 0.007910259470148999,
  'inhib': 0.0024890040574175733,
  'affect': 0.01827542705172355,
  'negemo': 0.005046200006819189,
  'anx': 0.0006478229738484094,
  'discrep': 0.0072283405503085684,
  'motion': 0.007330628388284633,
  'relativ': 0.058679123052269085,
  'conj': 0.024139929762351256,
  'excl': 0.014183913532680965,
  'pronoun': 0.03464148112789389,
  'ppron': 0.015786422994305976,
  'i': 0.0035800743291622625,
  'incl': 0.016093286508234172,
  'space': 0.024651368952231578,
  'ipron': 0.018855058133587916,
  'time': 0.027413140577585326,
  'adverb': 0.020184800027276757,
  'tentat': 0.013842954072760748,
  'work': 0.007

In [38]:
#correlation of LIWC features with label
liwc_pos_percent_df = pd.DataFrame(liwc_pos_percent).T
liwc_neg_percent_df = pd.DataFrame(liwc_neg_percent).T
liwc_pos_percent_df['label'] = 1
liwc_neg_percent_df['label'] = 0
liwc_percent_df = pd.concat([liwc_pos_percent_df, liwc_neg_percent_df])
liwc_percent_df = liwc_percent_df.fillna(0)
corr = liwc_percent_df.corr()


In [40]:
#choose the top 15 most correlated features to label
corr_label = corr['label'].sort_values(ascending=False)
corr_label = corr_label[1:16]
corr_label

i          0.360488
anx        0.358541
health     0.348380
sad        0.345127
affect     0.321898
negemo     0.320290
feel       0.278152
ppron      0.272873
adverb     0.249147
pronoun    0.248551
insight    0.213088
excl       0.205903
discrep    0.204839
cogmech    0.203785
filler     0.198220
Name: label, dtype: float64

In [ ]:
+